In [1]:

'''
We are now going to build the Agent
first,we load the index using the storage context and the load_index_from_storage function
'''

from llama_index.core import StorageContext,load_index_from_storage
from constant import embed_model

#load the index
storage_context = StorageContext.from_defaults(persist_dir='index/')
index = load_index_from_storage(storage_context,embed_model=embed_model) #we parse the storage context through the load_index_from_storage function and the embed_model 

In [2]:
'''
Create the RAG tool
This is the tool the agent will use to fetch information from the existing database
'''
from llama_index.core.tools import QueryEngineTool
from constant import llm_model


#load the llm model engine
query_engine= index.as_query_engine(llm_model=llm_model, similarity_top=5)
rag_tool=QueryEngineTool.from_defaults(
    query_engine,
    name = "research_paper_query_engine_tool",
    description = "A RAG engine with recent Research paper",
    )
    


In [3]:
#use this function to display the prompt

from IPython.display import Markdown,display

def display_prompt_dict(prompts_dict):
    for key, prompt in prompts_dict.items():
        display(Markdown(f"**Prompt key**: {key}"))
        print(prompt.get_template())

In [8]:
prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt key**: response_synthesizer:text_qa_template

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


**Prompt key**: response_synthesizer:refine_template

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 


In [9]:
#define the other tools the agent will use

from llama_index.core.tools import FunctionTool
from tools import download_pdf, fetch_arxiv_papers

download_pdf_tool = FunctionTool.from_defaults(
    download_pdf, 
    name="download_pdf_file_tool", 
    description="pyhton function that downloads a pdfs file by link",
    )

fetch_arxiv_tool = FunctionTool.from_defaults(
    fetch_arxiv_papers, 
    name="fetch_from_arxiv", 
    description="download the {max_result} recent papers regarding the {topicp} from arvix",
    )

In [11]:
#create the Agent(We use the React beacuse the agent will be able to react to the user input)
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [download_pdf_tool,rag_tool,fetch_arxiv_tool],llm=llm_model,verbose=True
)

In [12]:
'''
to be able to chat with the agent,we need a query template
'''
query_template = ''' I am interested in {topic}.
Find papers in your knowledge base that are related to {topic}.Use the following template to query research_paper_query_engine_tool:
Provide title,summary,authors and links to download to pappers related to {topic}.
if there are not,could you search the recent one from arxiv'''


In [14]:
answer = agent.chat(query_template.format(topic= "Multi-modal models"))

> Running step b796d6f6-eabe-4688-9e33-ad64e8d7be40. Step input:  I am interested in Multi-modal models.
Find papers in your knowledge base that are related to Multi-modal models.Use the following template to query research_paper_query_engine_tool:
Provide title,summary,authors and links to download to pappers related to Multi-modal models.
if there are not,could you search the recent one from arxiv
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Multi-modal models'}
Observation: Context information is below.
---------------------
Title: Distilling Multi-modal Large Language Models for Autonomous Driving
Authors: Deepti Hegde, Rajeev Yasarla, Hong Cai, Shizhong Han, Apratim Bhattacharyya, Shweta Mahajan, Litian Liu, Risheek Garrepalli, Vishal M. Patel, Fatih Porikli
summary: Autonomous driving demands safe motion planning, especially in critical
"long-tail" scen

In [15]:
#to better visualize the answer,we use the display function
Markdown(answer.response)

Here are some recent papers related to Multi-modal models:

1. **Title:** Distilling Multi-modal Large Language Models for Autonomous Driving  
   **Authors:** Deepti Hegde, Rajeev Yasarla, Hong Cai, Shizhong Han, Apratim Bhattacharyya, Shweta Mahajan, Litian Liu, Risheek Garrepalli, Vishal M. Patel, Fatih Porikli  
   **Summary:** This paper proposes DiMA, an end-to-end autonomous driving system that distills information from a multi-modal large language model (LLM) to a vision-based planner. It aims to improve efficiency while leveraging the world knowledge of an LLM, resulting in significant reductions in trajectory error and collision rates.  
   **Link to download:** [PDF](http://arxiv.org/pdf/2501.09757v1) | [arXiv](http://arxiv.org/abs/2501.09757v1)

2. **Title:** A Simple Aerial Detection Baseline of Multimodal Language Models  
   **Authors:** Qingyun Li, Yushi Chen, Xinya Shu, Dong Chen, Xin He, Yi Yu, Xue Yang  
   **Summary:** This paper presents a baseline for applying multimodal language models (MLMs) to aerial detection in remote sensing. It introduces a normalization method to transform detection outputs into textual outputs compatible with the MLM framework and achieves impressive detection performance.  
   **Link to download:** [PDF](http://arxiv.org/pdf/2501.09720v1) | [arXiv](http://arxiv.org/abs/2501.09720v1)

In [16]:
#since the agent retains memory,we can use the same query template to ask for another topic
answer = agent.chat("download all the papers you mentioned")

> Running step 32b8bf0c-1093-4378-bfe8-913127fb060d. Step input: download all the papers you mentioned
Thought: I need to download the PDF files of the papers related to Multi-modal models that I found earlier.
Action: download_pdf_file_tool
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2501.09757v1', 'output_file_name': 'Distilling_Multi-modal_Large_Language_Models_for_Autonomous_Driving.pdf'}
Observation: PDF file downloaded successfully to papers\Distilling_Multi-modal_Large_Language_Models_for_Autonomous_Driving.pdf
> Running step b75c3ed9-2ca4-483c-add3-4b0a6a7b035a. Step input: None
Thought: (Implicit) I can answer without any more tools!
Answer: Action: download_pdf_file_tool  
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2501.09720v1', 'output_file_name': 'A_Simple_Aerial_Detection_Baseline_of_Multimodal_Language_Models.pdf'}


In [17]:
Markdown(answer.response)

Action: download_pdf_file_tool  
Action Input: {'pdf_url': 'http://arxiv.org/pdf/2501.09720v1', 'output_file_name': 'A_Simple_Aerial_Detection_Baseline_of_Multimodal_Language_Models.pdf'}

In [18]:
#we want toask abot a topic which is not in the database
answer = agent.chat(query_template.format(topic= "Quantum Computing"))

> Running step 95056c1e-2fbe-4850-84b9-e0c3581afe09. Step input:  I am interested in Quantum Computing.
Find papers in your knowledge base that are related to Quantum Computing.Use the following template to query research_paper_query_engine_tool:
Provide title,summary,authors and links to download to pappers related to Quantum Computing.
if there are not,could you search the recent one from arxiv
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: research_paper_query_engine_tool
Action Input: {'input': 'Quantum Computing'}
Observation: Context information is below.
---------------------
Title: Inference-Time Scaling for Diffusion Models beyond Scaling Denoising Steps
Authors: Nanye Ma, Shangyuan Tong, Haolin Jia, Hexiang Hu, Yu-Chuan Su, Mingda Zhang, Xuan Yang, Yandong Li, Tommi Jaakkola, Xuhui Jia, Saining Xie
summary: Generative models have made significant impacts across various domains,
largely due to their ability t

In [19]:
Markdown(answer.response)

I couldn't find any papers related to Quantum Computing in the current knowledge base or recent searches. You may want to check platforms like arXiv directly for the latest research in this area.